In [1]:
import requests
import pandas as pd # 0.23.0
import folium
from folium import FeatureGroup, features, CircleMarker
from haversine import haversine
from os import listdir
from os.path import isfile, join
import numpy as np

In [2]:
def popup(i,l,t,show=False):
    return folium.Popup('#%d [%f,%f]\n%s' % (i,l[0],l[1],str(t)),show=show)

Get all files in the directory with the downloaded files

In [3]:
mypath='./20190731'
fls=[f for f in listdir(mypath) if isfile(join(mypath, f))]

In [4]:
racers = pd.read_pickle('racers.pkl')

I tested the query in http://overpass-turbo.eu/ and got the bounding box from there. From there: 'Export->Query->Standalone Query'

In [5]:
query =\
"""
[out:json][timeout:900][bbox:42.68757692735903,21.614227294921875,43.191245629858116,22.17041015625];
way[highway=motorway];
(._;>;);
out;
"""

Retrieve the data from OSM

In [6]:
r = requests.get('http://overpass-api.de/api/interpreter?data='+query)
print(r.status_code)
ores = r.json()

200


I just collected the location data from the returned JSON file. There is much more data available, for example street signs.

In [7]:
a1 =[(e['lat'],e['lon']) for e in ores['elements'] if e['type']=='node']

Display the section I am looking at.

In [8]:
m = folium.Map(tiles='OpenStreetMap')
fg = folium.FeatureGroup()
[folium.CircleMarker(location=[e[0],e[1]],radius=1,color='red').add_to(m) for e in a1]
m.fit_bounds(m.get_bounds())
m

Short routine to calculate the distance from the A1, collect the values less then `mindist`. `haversin`is the function to calculate the distance from to geo location points, see https://pypi.org/project/haversine/

In [9]:
def onRoad(a1, racer,mindist=50):
    ll =[l for i,(l,t) in enumerate(pd.read_pickle(mypath+'/'+racer).values.tolist())]
    ilast =np.argmin([haversine(a1[0],l,unit='m') for l in ll])+2
    ifirst =np.argmin([haversine(a1[-1],l,unit='m') for l in ll])-2
    ddll=[]
    for i,l in enumerate(ll[ifirst:ilast]):
        dd = [haversine(a1,l,unit='m') for a1 in a1]
        id = np.argmin(dd)
        if dd[id] < mindist:
              ddll.append([i+ifirst,l,a1[id],dd[id]])
    return ddll            

After some trial and error testing I used a `mindst` of 10m.

In [10]:
res = [[racer.split('.')[0],onRoad(a1,racer,10)] for racer in fls]

The number of files I looked at:

In [11]:
len(res)

254

There is a section where the next offical road is very close and there is a bridge. So there might be trackpoints on the A1 even if the rider is not on the A1. Other might have left the A1 shortly after they noticed their error. So I only looked at the riders with more then two track points on or very near the A1 location points.

In [12]:
bg = [r for r in res if len(r[1])>2]
bg

[['Control_Car_1',
  [[258, [42.86654, 22.10762], (42.8665678, 22.107574), 4.859056869750684],
   [259, [42.91865, 22.03291], (42.9186019, 22.032906), 5.358392409483363],
   [260, [42.99539, 22.02579], (42.9953744, 22.0257586), 3.087156368310005],
   [261, [43.05012, 21.983], (43.0501164, 21.9829251), 6.099267250101745],
   [264, [43.18048, 21.86329], (43.1804718, 21.863267), 2.075890332431289]]],
 ['Valero_Puchades',
  [[188, [43.11103, 21.92517], (43.1110069, 21.925095), 6.607856851739296],
   [190, [43.14491, 21.89938], (43.1448827, 21.8992821), 8.50304278239754],
   [191, [43.16465, 21.88116], (43.1646365, 21.8810534), 8.775109777585095]]],
 ['Osamu_Brown',
  [[595, [42.69936, 22.07613], (42.6993231, 22.0760514), 7.621847808429807],
   [596, [42.71599, 22.07477], (42.7160644, 22.0747049), 9.83503055785059],
   [602, [42.8446, 22.13142], (42.8445675, 22.131325), 8.54680298505161],
   [627, [43.1141, 21.92167], (43.1140177, 21.9216443), 9.386116233201774],
   [630, [43.15365, 21.8932

In [13]:
racers[racers.Name.isin([e[0] for e in bg])]

,Name,Bib
0,Control_Car_1,CC1
144,Osamu_Brown,48
246,Valero_Puchades,226


## Control Car

In [14]:
f = bg[0][0]
m = folium.Map(tiles='OpenStreetMap',zoom_start=10)
[folium.CircleMarker(location=l,radius=2,color='blue',fill=True,popup=popup(i,l,t)).add_to(m)
 for i,(l,t) in enumerate(pd.read_pickle(mypath+'/'+f+'.pkl').values.tolist()) if i in range(min([b[0] for b in bg[0][1]]),max([b[0] for b in bg[0][1]]))]
m.fit_bounds(m.get_bounds())
m

## Valero_Puchades

In [15]:
ib = 1
f = bg[ib][0]
m = folium.Map(tiles='OpenStreetMap',zoom_start=10)
fg = folium.FeatureGroup()
#[folium.CircleMarker(location=[e['lat'],e['lon']],radius=1,color='lightblue').add_to(m) for e in ores['elements'] if e['type']=='node']
[folium.CircleMarker(location=l,radius=2,color='blue',fill=True,popup=popup(i,l,t)).add_to(fg)
 for i,(l,t) in enumerate(pd.read_pickle(mypath+'/'+f+'.pkl').values.tolist()) if i in range(min([b[0] for b in bg[ib][1]]),max([b[0] for b in bg[ib][1]]))]
m.add_child(fg)
m.fit_bounds(fg.get_bounds())
m

## Osamu_Brown

You have to zoom into the end of the section to see that he used the A1.

In [16]:
ib = 2
f = bg[ib][0]
m = folium.Map(tiles='OpenStreetMap',zoom_start=10)
fg = folium.FeatureGroup()
#[folium.CircleMarker(location=[e['lat'],e['lon']],radius=1,color='lightblue').add_to(m) for e in ores['elements'] if e['type']=='node']
[folium.CircleMarker(location=l,radius=2,color='blue',fill=True,popup=popup(i,l,t)).add_to(fg)
 for i,(l,t) in enumerate(pd.read_pickle(mypath+'/'+f+'.pkl').values.tolist()) if i in range(min([b[0] for b in bg[ib][1]]),max([b[0] for b in bg[ib][1]]))]
m.add_child(fg)
m.fit_bounds(fg.get_bounds())
m